<a href="https://colab.research.google.com/github/priyatidke/pytorchtoTflite/blob/main/Rapid_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Machine Learning / Deep Learning Engineer Evaluation Task**

**Goal:**

The goal of this task to be able to run a pose estimation model in C++ using Tensorflow Lite C++ API.

**Task 1:** Convert the Pytorch model from pose_hrnet_w32_256x256.pth to TensorFlow Lite model







In [1]:
#To check Tensorflow version
import tensorflow
print(tensorflow.__version__)

2.4.1


In [2]:
#GPU device
import tensorflow as tf

devices = tf.config.experimental.list_physical_devices(device_type="GPU")
devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
#install onnx_tf 
!pip install onnx_tf

In [4]:
!pip install onnx

In [5]:
# Accessing My Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import required libraries**

In [6]:
#import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import onnx
from onnx import backend
from onnx_tf.backend import prepare
import tensorflow as tf


In [7]:
#set path for file
PATH = "drive/My Drive/POSE/pose_hrnet_w32_256x256.pth"


**Define SuperResolutionNet model**



In [8]:
#Base model
#SuperResolutionNet model

class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(3, 64, (3, 3), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        #self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

In [9]:
# Create the super-resolution model by using the above model definition.
model = SuperResolutionNet(upscale_factor=3)

# Initialize model with the pretrained weights
model.load_state_dict(torch.load(PATH), strict=False)

# set the model to inference mode
model.eval()

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle(upscale_factor=3)
)

In [10]:
#Generate and pass random input so the Pytorch exporter can trace the model and save it to an ONNX file

dummy_input = Variable(torch.randn(1, 3, 224, 224))
torch.onnx.export(model, dummy_input, "drive/My Drive/POSE/hrnet_new.onnx")

In [11]:
#Load the ONNX file 
onnx_model = onnx.load('drive/My Drive/POSE/hrnet_new.onnx')

#import ONNX model to Tensorflow
tf_rep = prepare(onnx_model) 

In [12]:
#inputs node to the model
print("inputs:", tf_rep.inputs)

inputs: ['input.1']


In [13]:
#outputs node from the model
print("outputs:", tf_rep.outputs)

outputs: ['20']


In [14]:
#All nodes in the model
print("tensor_dict")
print(tf_rep.tensor_dict)

tensor_dict
{}


In [15]:
# exporting to TensorFlow graphs
tf_rep.export_graph('drive/My Drive/POSE/hrnet_new.pb')

INFO:tensorflow:Assets written to: drive/My Drive/POSE/hrnet_new.pb/assets


INFO:tensorflow:Assets written to: drive/My Drive/POSE/hrnet_new.pb/assets


**TensorFlow Model to TFLite**

In [16]:
#load TensorFlow model from saved_model
loaded = tf.saved_model.load('drive/My Drive/POSE/hrnet_new.pb')

In [17]:
#Concrete Function
concrete_func = loaded.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

concrete_func.inputs[0].set_shape([1, 3, 224, 224])

#To load the concrete function into the TFLiteConverter

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [18]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]


In [19]:
#converte TensorFlow Lite model
tf_lite_model = converter.convert()


In [20]:
#Store the Model
open('drive/My Drive/POSE/hrnet_new.tflite', 'wb').write(tf_lite_model)

print("Converted to tensorflow lite succesfully.")

Converted to tensorflow lite succesfully.


**Inference with TFLite model**









In [21]:
interpreter = tf.lite.Interpreter(model_path='drive/My Drive/POSE/hrnet_new.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
print("input_details",input_details)
output_details = interpreter.get_output_details()  
print("output_details", output_details)      
input_shape = input_details[0]['shape']
print("input_shape", input_shape)      

input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print("input_data", input_data)      

interpreter.set_tensor(input_details[0]['index'], input_data)


input_details [{'name': 'input_1', 'index': 0, 'shape': array([  1,   3, 224, 224], dtype=int32), 'shape_signature': array([  1,   3, 224, 224], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output_details [{'name': 'Identity', 'index': 28, 'shape': array([  1,   1, 678, 678], dtype=int32), 'shape_signature': array([  1,   1, 678, 678], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
input_shape [  1   3 224 224]
input_data [[[[0.7076335  0.5023663  0.5260045  ... 0.8175962  0.19163184
    0.4107162 ]
   [0.34354818 0.76715    0.8471102  ... 0.94694155 0.94958943
    0.85409456]
   [0.46885368 0.12112184 0.16566078 ... 0.6

In [22]:
#interpreter.invoke()

#y = interpreter.get_tensor(output_details[0]['index'])    
#feature =  interpreter.get_tensor(output_details[1]['index'])